In [1]:
import numpy as np
import pandas as pd
import pdb
import pickle
import scipy
from sklearn.preprocessing import OneHotEncoder
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
CORPUS_PATH = '/kaggle/input/pahaw-dataset/PaHaW/PaHaW_files/corpus_PaHaW.xlsx'
SVC_DIR = '/kaggle/input/pahaw-dataset/PaHaW/PaHaW_public'
IMG_DIM = (6000, 10000)
TINY_DIM = (300, 500)

In [3]:
corpus = pd.read_excel(CORPUS_PATH)
corpus.head()

,ID,Nationality,Sex,Disease,PD status,Age,Dominant hand,LED,UPDRS V,Length of PD
0,1,Czech,F,PD,ON,68,R,1115.0,2.0,6.0
1,2,Czech,F,PD,ON,78,R,2110.0,2.0,8.0
2,3,Czech,F,PD,ON,69,R,1556.6,2.0,7.0
3,4,Czech,F,PD,ON,79,R,1691.0,2.0,12.0
4,5,Czech,F,PD,ON,69,R,600.0,2.0,2.0


In [4]:
class Subject(object):
    
    pass

In [5]:
class PaHaWDataset(object):
    """A dataset object"""
    def __init__(self, subjects):
        self.subjects = subjects

        # Default values
        self._maxlen = None
        self._method = None
        self.subsample_rate = 100
        self.window_size = 100
        self.stride = 20

    def subsample(self, task):
        """Subsample rows from a task"""
        return task[::self.subsample_rate, :]

    def window(self, task):
        """Generate windows from a task"""
        n_windows = (task.shape[0] - self.window_size) // self.stride + 1
        return [
            task[(i*self.stride):(i*self.stride + self.window_size), :] for i in range(n_windows)
        ]

    def summarize(self, task):
        """Convert sequence of task values into summary stat==tics"""
        in_air = task[:, 3] == 0
        try:
            summary_vect = np.array([
                np.std(task[in_air, 13]), # Std. dev. of in air velocity
                np.min(task[in_air, 11]), # Min of vertical jerk
                np.std(task[in_air, 14]), # Std. dev. of in air accel.
                # Range of horizontal jerk
                np.max(task[in_air, 12]) - np.min(task[in_air, 12]),
                np.std(task[in_air, 15]), # Std. dev of in air jerk
                # Range of horizontal accel.
                np.max(task[in_air, 10]) - np.min(task[in_air, 10]),
                # Range of horizontal velocity.
                np.max(task[in_air, 8]) - np.min(task[in_air, 8]),
                # 75th percentile of on-surface horizontal velocity
                np.percentile(task[np.logical_not(in_air), 8], 0.75),
                np.min(task[in_air, 10]), # Min. in air vertical accel.
                # 99-1 percentile vertical velocity
                np.percentile(task[in_air, 8], 0.99) - np.percentile(task[in_air, 8], 0.01),
                # Mean velocity
                np.mean(task[in_air, 13]),
                # Mean altitutde velocity
                np.mean(task[in_air, 17]),
                # 99-1 percentile altitude velocity
                np.percentile(task[in_air, 17], 0.99) - np.percentile(task[in_air, 17], 0.01),
                # Std dev. altitude velocity
                np.std(task[in_air, 17])

            ])
            return summary_vect
        except:
            # If there are ==sues generating summary stats it == probably
            # because pen was never lifted from paper - e.g. in task 1. We will
            # just skip the task if th== happens.
            pass

In [6]:
class PaHaWDataset(object):
    """A dataset object"""
    def __init__(self, subjects):
        self.subjects = subjects

        # Default values
        self._maxlen = None
        self._method = 'img'
#         self.method = 'img'
        self.subsample_rate = 100
        self.window_size = 100
        self.stride = 20

    def subsample(self, task):
        """Subsample rows from a task"""
        return task[::self.subsample_rate, :]

    def window(self, task):
        """Generate windows from a task"""
        n_windows = (task.shape[0] - self.window_size) // self.stride + 1
        return [
            task[(i*self.stride):(i*self.stride + self.window_size), :] for i in range(n_windows)
        ]

    def summarize(self, task):
        """Convert sequence of task values into summary statistics"""
        in_air = task[:, 3] == 0
        try:
            summary_vect = np.array([
                np.std(task[in_air, 13]), # Std. dev. of in air velocity
                np.min(task[in_air, 11]), # Min of vertical jerk
                np.std(task[in_air, 14]), # Std. dev. of in air accel.
                # Range of horizontal jerk
                np.max(task[in_air, 12]) - np.min(task[in_air, 12]),
                np.std(task[in_air, 15]), # Std. dev of in air jerk
                # Range of horizontal accel.
                np.max(task[in_air, 10]) - np.min(task[in_air, 10]),
                # Range of horizontal velocity.
                np.max(task[in_air, 8]) - np.min(task[in_air, 8]),
                # 75th percentile of on-surface horizontal velocity
                np.percentile(task[np.logical_not(in_air), 8], 0.75),
                np.min(task[in_air, 10]), # Min. in air vertical accel.
                # 99-1 percentile vertical velocity
                np.percentile(task[in_air, 8], 0.99) - np.percentile(task[in_air, 8], 0.01),
                # Mean velocity
                np.mean(task[in_air, 13]),
                # Mean altitutde velocity
                np.mean(task[in_air, 17]),
                # 99-1 percentile altitude velocity
                np.percentile(task[in_air, 17], 0.99) - np.percentile(task[in_air, 17], 0.01),
                # Std dev. altitude velocity
                np.std(task[in_air, 17])

            ])
            return summary_vect
        except:
            # If there are issues generating summary stats it is probably
            # because pen was never lifted from paper - e.g. in task 1. We will
            # just skip the task if this happens.
            pass

    def extract_imgs(self, task):
        # Initialize image arrays
        img_paper = np.zeros(IMG_DIM)
        img_air = np.zeros(IMG_DIM)
        # Extract coordinates
        x = task[:, 1]
        y = task[:, 0]
        idx = task[:, 3]
        # Upsample handwriting samples
        t = np.arange(task.shape[0])
        dt = np.linspace(0, task.shape[0], 100*task.shape[0])
        x = np.interp(dt, t, x).astype(np.int16)
        y = np.interp(dt, t, y).astype(np.int16)
        idx = np.interp(dt, t, idx).astype(np.int16)
        # Seperate on-paper and in-air samples
        on_paper = idx == 1
        in_air = idx == 0
        # Create thickened images
        n = 5
        for dx in range(-n, n+1):
            for dy in range(-n, n+1):
                img_paper[y[on_paper] + dy, x[on_paper] + dx] += 1
                img_air[y[in_air] + dy, x[in_air] + dx] += 1
        # Downsample images
        img_paper = img_paper[::20, ::20]
        img_air = img_air[::20, ::20]
        return img_paper, img_air

    def update(self):
        from keras.preprocessing import sequence
        x = []
        y = []

        # Process the task data according to active method for each subject.
        # Append results to x and y
        for subject in self.subjects:
            if self.method == 'subsample':
                tasks = [self.subsample(task) for task in subject.task.itervalues()]
            if self.method == 'window':
                tasks = []
                for task in subject.task.itervalues():
                    tasks += self.window(task)
            if self.method is None:
                tasks = subject.task.values()
            if self.method == 'summary':
                tasks = [self.summarize(task) for task in
                         subject.task.itervalues()]
                tasks = [task for task in tasks if task is not None]
            if self.method == 'img':
                print(subject)
                img_paper = np.zeros(TINY_DIM)
                img_air = np.zeros(TINY_DIM)
                for task in subject.task.values():
                    dp, da = self.extract_imgs(task)
                    img_paper += dp
                    img_air += da
                tasks = [(img_paper, img_air)]
            x += tasks
            y += [subject.info['PD status']] * len(tasks)

        # Convert to arrays
        if self.method == 'summary':
            x = np.vstack(x)
        elif self.method == 'img':
#             pdb.set_trace()
            on_paper = np.stack([task[0].reshape(1, TINY_DIM[0], TINY_DIM[1])  for task in x])
            in_air = np.stack([task[1].reshape(1, TINY_DIM[0], TINY_DIM[1]) for task in x])
            x = on_paper, in_air
        else:
            x = sequence.pad_sequences(x, maxlen=400, dtype='float32')

        y = np.array(y, dtype='float32')

        # Normalize x-values
        # x = x / np.max(x, axis=(0,1))

        # Assign attributes
        self.x = x
        self.y = y

    @property
    def method(self):
        return self._method

    @method.setter
    def method(self, method):
        if method in ['subsample', 'window', 'summary', 'img', None]:
            self._method = method
            self.update()
        else:
            raise ValueError('Invalid method')

    @property
    def maxlen(self):
        return self._maxlen

    @maxlen.setter
    def maxlen(self, maxlen):
        self._maxlen = maxlen
        self.update()

    @property
    def subsample_rate(self):
        return self._subsample_rate

    @subsample_rate.setter
    def subsample_rate(self, subsample_rate):
        if type(subsample_rate) is int:
            self._subsample_rate = subsample_rate
            self.update()
        else:
            raise ValueError('Subsample rate must be an int')

    @property
    def window_size(self):
        return self._window_size

    @window_size.setter
    def window_size(self, window_size):
        if type(window_size) is int:
            self._window_size = window_size
            self.update()
        else:
            raise ValueError('Window size must be an int')

    @property
    def stride(self):
        return self._stride

    @stride.setter
    def stride(self, stride):
        if type(stride) is int:
            self._stride = stride
            self.update()
        else:
            raise ValueError('Stride must be an int')

In [7]:
def parse_corpus(path):
    
    df = pd.read_excel(path)
    # Replace PD status with a binary variable.
    df['PD status'] = df['PD status'] == 'ON'
    df['PD status'] = df['PD status'].astype(int)
    # --- ADD OTHER PROCESSING STEPS HERE ---

    return df

In [8]:
def generate_svc_path(subject_id, task_index):

    return "%s/%s/%s__%i_1.svc" % (SVC_DIR, subject_id, subject_id, task_index)


In [9]:
def parse_svc(path, i):

    # Open the file
    with open(path, 'r') as svc_file:
        samples = svc_file.readlines()

    # Extract the data
    data = []
    for sample in samples[1:]:
        values = [int(value) for value in sample.split()]
        data.append(values)
    array = np.array(data)

    # ---Process the data---

    # remove time stamp
    # array = np.delete(array, 2, axis=1)
    n = array.shape[0]

    # position based velocity, acceleration, and jerk
    xy_vel = displace(array[:,0:2])
    xy_accel = displace(xy_vel)
    xy_jerk = displace(xy_accel)

    # magnitudes of previous measurements
    m_vel = np.linalg.norm(xy_vel, axis=1).reshape((n, 1))
    m_accel = np.linalg.norm(xy_accel, axis=1).reshape((n, 1))
    m_jerk = np.linalg.norm(xy_jerk, axis=1).reshape((n, 1))

    # rate of change of azimuth, altitude, and pressure
    aap_vel = displace(array[:,4:])

    # add task id as feature
    # print array.shape
    new_col = (i-1) * np.ones((n,1))
#     enc = OneHotEncoder(n_values=8)
    # one_hot = np.asarray(enc.fit_transform(new_col).todense())

    out = np.concatenate((array, xy_vel, xy_accel, xy_jerk, m_vel, m_accel,
                          m_jerk, aap_vel), axis=1)
    return out

In [10]:
def displace(array):
    """Generates displacement vectors for columns in an array.

    Arg:
        array: np.array. Vectors to be displaced.
    """
    disp = np.zeros(shape=array.shape)
    disp[1:,:] = array[1:,:] - array[0:-1,:]
    return disp

In [11]:
def extract_datasets(test_fraction = 0.333):
    """Extracts the train and test datasets.

    Returns:
        train, test: PaHaWDataset() objects.
    """
    import random
    #train_subjects = []
    #test_subjects = []
    subjects = []

    # Extract data from corpus
    corpus = parse_corpus(CORPUS_PATH)

    # Build a Subject object for each row in the corpus and randomly assign to
    # train or test dataset
    for i, row in enumerate(corpus.iterrows()):
        subject = Subject()
        subject.info = row[1]
        subject.task = dict()
        # ID must be converted from int to fixed length string.
        subject_id = '%05d' % row[1].ID
        # Extract task data from SVC files
        for i in range(1, 9):
            try:
                svc_path = generate_svc_path(subject_id, i)
                task_data = parse_svc(svc_path, i)
                subject.task[i] = task_data
            except IOError:
                print (f'Subject {subject_id} did not perform task {i}') 
        # if random.random() < test_fraction:
        #     test_subjects.append(subject)
        # else:
        #     train_subjects.append(subject)
        subjects.append(subject)
    # Load subjects into datasets
    # train = PaHaWDataset(train_subjects)
    # test = PaHaWDataset(test_subjects)
    data = PaHaWDataset(subjects)

    # return train, test
    return data

In [12]:
data = extract_datasets()

Subject 00061 did not perform task 1
Subject 00080 did not perform task 1
Subject 00089 did not perform task 1


In [13]:
with open('/kaggle/working/processed_img_data.pkl', 'wb') as pkl_file:
        pickle.dump(data, pkl_file)

In [14]:
with open('/kaggle/working/processed_img_data.pkl', 'rb') as pkl_file:
        data = pickle.load(pkl_file)

In [15]:
image_paper, image_air = data.x
labels = data.y

In [16]:
image_paper.shape, image_air.shape, labels.shape

((75, 1, 300, 500), (75, 1, 300, 500), (75,))

In [17]:
image_paper = image_paper.reshape(75, 300, 500, 1)

In [18]:
# K.set_image_dim_ordering('th')

batch_size = 4
nb_folds = 10
nb_epoch = 30

img_rows = 300
img_cols = 500

mdl_type = 'air'

In [19]:
def make_model():
    model = Sequential()
 
    model.add(Conv2D(8, (4, 4), input_shape=(img_rows, img_cols, 1), kernel_regularizer=l2(0.001)))
    model.add(Activation('relu'))
    model.add(Conv2D(8, (4, 4), kernel_regularizer=l2(0.001)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    
    model.add(Flatten())
    model.add(Dense(8, kernel_regularizer=l2(0.001)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [20]:
model = make_model()
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 297, 497, 8)    │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 297, 497, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 294, 494, 8)    │         1,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 294, 494, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 147, 247, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 147, 247, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 290472)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │     2,323,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,324,961 (8.87 MB)

 Trainable params: 2,324,961 (8.87 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
train_datagen = ImageDataGenerator(
    rotation_range=2.5,
    width_shift_range=0.025,
    height_shift_range=0.025,
    zoom_range=0.025
)

In [22]:
valid_datagen = ImageDataGenerator(
    preprocessing_function=None 
)

In [23]:
n_samples = len(labels)
fold_size = n_samples // nb_folds
idx = np.arange(n_samples)
np.random.shuffle(idx)
import os
os.makedirs('./saved_models', exist_ok=True)
hists = {}

# timestamp = time.strftime("%y-%m-%d_%H-%M")

for k in range(nb_folds):
    print (f"On fold {k}") 
    valid_idx = idx[k*fold_size:(k+1)*fold_size]
    train_idx = np.setdiff1d(idx, valid_idx)

    x_train = image_paper[train_idx]
    x_valid = image_paper[valid_idx]
    y_train = labels[train_idx]
    y_valid = labels[valid_idx]

        #fit feature generator
    train_datagen.fit(x_train)

    print ('Training model')
    model = make_model()
    hist = model.fit(
        train_datagen.flow(x_train, y_train, batch_size=batch_size),
        
        validation_data = (x_valid, y_valid),
        epochs=nb_epoch)
    
    
    model.save(f'saved_models/model_{mdl_type}_fold_{k}.keras')
    hists[k] = hist.history

with open(f'saved_models/model_{mdl_type}_hists.pkl', 'wb') as pkl_file:
    pickle.dump(hists, pkl_file)

On fold 0
Training model
Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 10s 440ms/step - accuracy: 0.6025 - loss: 2.7773 - val_accuracy: 0.8571 - val_loss: 0.3307
Epoch 2/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 384ms/step - accuracy: 0.6203 - loss: 0.7698 - val_accuracy: 0.8571 - val_loss: 0.4422
Epoch 3/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 380ms/step - accuracy: 0.6578 - loss: 0.6553 - val_accuracy: 0.8571 - val_loss: 0.4502
Epoch 4/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 10s 396ms/step - accuracy: 0.6253 - loss: 0.6847 - val_accuracy: 0.8571 - val_loss: 0.5596
Epoch 5/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 10s 405ms/step - accuracy: 0.4871 - loss: 0.7381 - val_accuracy: 0.7143 - val_loss: 0.5302
Epoch 6/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 372ms/step - accuracy: 0.6597 - loss: 0.6537 - val_accuracy: 0.8571 - val_loss: 0.5034
Epoch 7/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 10s 377ms/step - accuracy: 0.7164 - loss: 0.6652 - val_accuracy: 0.7143 - val_loss: 0.4953
Epoch 8/30
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 377ms/step - accuracy: 0.6302 - loss: 0.6992 - val_accuracy: 0.7143 - 